In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 5

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../training/PlantVillage",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)


Found 2152 files belonging to 3 classes.


In [4]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [5]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

(32, 256, 256, 3)
[1 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1 1 2 0 0]


In [6]:
len(dataset)

68

In [7]:
train_size = 0.8
len(dataset)*train_size


54.400000000000006

In [8]:
train_ds = dataset.take(54)
len(train_ds)


54

In [9]:
test_ds = dataset.skip(54)
len(test_ds)


14

In [10]:
val_size = 0.1
len(dataset)*val_size


6.800000000000001

In [11]:
val_ds = test_ds.take(6)
len(val_ds)


6

In [12]:
test_ds = test_ds.skip(6)
len(test_ds)


8

In [13]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds


In [14]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)


In [15]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)


In [16]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1./255),
])


In [17]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1./255),
])


In [18]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])


In [19]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)


In [20]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu',
                  input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)


In [21]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 60, 60, 64)         

In [22]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


In [30]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50
)


Epoch 1/50
54/54 [==============================] - 61s 947ms/step - loss: 0.8801 - accuracy: 0.5625 - val_loss: 0.7235 - val_accuracy: 0.6823
Epoch 2/50
54/54 [==============================] - 46s 841ms/step - loss: 0.6007 - accuracy: 0.7413 - val_loss: 0.6748 - val_accuracy: 0.6823
Epoch 3/50
54/54 [==============================] - 44s 804ms/step - loss: 0.4469 - accuracy: 0.8056 - val_loss: 0.5084 - val_accuracy: 0.7865
Epoch 4/50
54/54 [==============================] - 46s 856ms/step - loss: 0.3070 - accuracy: 0.8767 - val_loss: 0.2659 - val_accuracy: 0.8698
Epoch 5/50
54/54 [==============================] - 51s 939ms/step - loss: 0.2848 - accuracy: 0.8837 - val_loss: 0.2181 - val_accuracy: 0.8906
Epoch 6/50
54/54 [==============================] - 51s 939ms/step - loss: 0.1901 - accuracy: 0.9259 - val_loss: 0.3046 - val_accuracy: 0.8542
Epoch 7/50
54/54 [==============================] - 45s 835ms/step - loss: 0.1987 - accuracy: 0.9253 - val_loss: 0.2129 - val_accuracy: 0.9062

In [31]:
scores = model.evaluate(test_ds)

8/8 [==============================] - 4s 268ms/step - loss: 0.0832 - accuracy: 0.9727


In [32]:
scores


[0.08318143337965012, 0.97265625]

In [33]:
history


In [34]:
history.params


{'verbose': 1, 'epochs': 50, 'steps': 54}

In [35]:
history.history.keys()


dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [36]:
type(history.history['loss'])


list

In [37]:
len(history.history['loss'])


50

In [38]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


In [39]:
import numpy as np


def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
#     predicted_class = class_names[predictions[0]]
    confidence = round(100 * (np.max(predictions[0])), 2)
#     confidence = round(100 * (predictions[0]), 2)
    return predicted_class, confidence


In [40]:
for images, labels in test_ds.take(1):
    for i in range(9):
        #         ax = plt.subplot(3, 3, i + 1)
        #         plt.imshow(images[i].numpy().astype("uint8"))

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]

        print(
            f"Actual: {actual_class}, \nPredicted: {predicted_class}.\n Confidence: {confidence}%")
#         f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")

#         plt.axis("off")


1/1 [==============================] - 0s 296ms/step
Actual: Potato___healthy, 
Predicted: Potato___Late_blight.
 Confidence: 78.5%
1/1 [==============================] - 0s 43ms/step
Actual: Potato___Early_blight, 
Predicted: Potato___Early_blight.
 Confidence: 100.0%
1/1 [==============================] - 0s 46ms/step
Actual: Potato___Late_blight, 
Predicted: Potato___Late_blight.
 Confidence: 100.0%
1/1 [==============================] - 0s 40ms/step
Actual: Potato___Early_blight, 
Predicted: Potato___Early_blight.
 Confidence: 100.0%
1/1 [==============================] - 0s 46ms/step
Actual: Potato___Early_blight, 
Predicted: Potato___Early_blight.
 Confidence: 99.98%
1/1 [==============================] - 0s 45ms/step
Actual: Potato___Early_blight, 
Predicted: Potato___Early_blight.
 Confidence: 100.0%
1/1 [==============================] - 0s 40ms/step
Actual: Potato___Late_blight, 
Predicted: Potato___Late_blight.
 Confidence: 100.0%
1/1 [==============================] - 0s 42

In [43]:
model.save("Model1.h5")
# reconstructed_model = tf.keras.models.load_model("my_h5_model.h5")

In [44]:
# MODEL = tf.keras.models.load_model('../saved_models/1')
MODEL = tf.keras.models.load_model('Model1.h5')

In [32]:
# MODEL1 = tf.keras.models.load_model('../saved_models/')

In [26]:
import tensorflow as tf
print(tf. __version__)


2.9.1


In [27]:
# from fastapi import FastAPI, File, UploadFile
# import uvicorn
# import numpy as np
# from io import BytesIO
# from PIL import Image
# import tensorflow as tf
# import asyncio
# import requests

from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf
import requests
from requests import request
from keras.preprocessing import image
import json
import base64


In [28]:
app = FastAPI()
# import os
# os.environ['PATH']

In [ ]:
# origins = [
#     "http://localhost",
#     "http://localhost:3000",
# ]
# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=origins,
#     allow_credentials=True,
#     allow_methods=["*"],
#     allow_headers=["*"],
# )

# MODEL = tf.keras.models.load_model("../saved_models/1")

CLASS_NAMES = ["Early Blight", "Late Blight", "Healthy"]

@app.get("/ping")
async def ping():
    return "Hello, I am alive"

def read_file_as_image(data) -> np.ndarray:
    image = np.array(Image.open(BytesIO(data)))
    return image

@app.post("/predict")
async def predict(
    file: UploadFile = File(...)
):

    image = read_file_as_image(await file.read())
    img_batch = np.expand_dims(image, 0)
    
    predictions = MODEL.predict(img_batch)

    predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
    confidence = np.max(predictions[0])
    return {
        'class': predicted_class,
        'confidence': float(confidence)
    }


# how to send image file inside request in python

# @app.post("/uploadfile")
# async def create_upload_file(file: UploadFile):
#     return {"filename": file.filename}


# extra

# @app.post("/upload")
# def upload(file: UploadFile = File(...)):
#     try:
#         contents = file.file.read()
#         with open(file.filename, 'wb') as f:
#             f.write(contents)
#     except Exception:
#         return {"message": "There was an error uploading the file"}
#     finally:
#         file.file.close()

#     return {"message": f"Successfully uploaded {file.filename}"}



# if __name__ == "__main__":
#     config = uvicorn.Config(app)
#     server = uvicorn.Server(config)
#     await server.serve()

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()
    # uvicorn.run(app, host='localhost', port=8000)


In [ ]:
# from fastapi import FastAPI, File, UploadFile, responses

# from fastapi import FastAPI, File, UploadFile
# from fastapi.middleware.cors import CORSMiddleware
# import uvicorn
# import numpy as np
# from io import BytesIO
# from PIL import Image
# import tensorflow as tf
# import requests
# from requests import request
# from keras.preprocessing import image
# # from deta import Drive

# app = FastAPI()
# # files = Drive("files")


# @app.post("/")
# def upload(file: UploadFile = File(...)):
#     # return files.put(file.filename, file.file)
#     return file.filename

# if __name__ == "__main__":
#     config = uvicorn.Config(app)
#     server = uvicorn.Server(config)
#     await server.serve()